In [11]:
function A(N, i,mu,t)
    sum(1:N-1) do k
        _cs_term(N, i,mu,k,t)
    end
end

function _cs_term(N, i,mu,k,t)
    cos(pi*(i-0.5)*k / N) * cos(pi*(mu-0.5)*k / N) * cos(pi*k /N/2)^2t 
end



0.12500000000000006

In [12]:
function sumAA(N, i,j,t)
    sum(1:N) do mu
        A(N, i,mu,t) * A(N, j,mu,t)
    end
end

function rhs(N, i,j,t)
    N/2*sum(1:N) do eta
        cos(pi*(i-0.5)*eta / N) * cos(pi*(j-0.5)*eta / N) * cos(pi*eta /N/2)^4t
    end
end

rhs (generic function with 1 method)

In [14]:
N, i, j, t = 40,3,2,2

sumAA(N, i,j,t), rhs(N, i,j,t)

(79.0625, 79.06250000000001)

In [7]:
str2bin = Dict('I' => (false, false), 'X' => (true, false), 'Y' => (true, true), 'Z' => (false, true))

Dict{Char, Tuple{Bool, Bool}} with 4 entries:
  'I' => (0, 0)
  'Y' => (1, 1)
  'Z' => (0, 1)
  'X' => (1, 0)

In [9]:
str2bin['I']

(false, false)

In [6]:
name  = "IXY"

for p in name
    v = 
end

IXY

In [19]:
v = [str2bin[p] for p in name]
v = (Tuple ∘ collect ∘ Iterators.flatten)(v)


(false, false, true, false, true, true)

In [20]:
typeof(v)

NTuple{6, Bool}

In [17]:
typeof((0,0,0,0)) 

NTuple{4, Int64}

In [1]:
module Paulis
    export Pauli, *
    
    
    str2bin = Dict('I' => (false, false), 'X' => (true, false), 'Y' => (true, true), 'Z' => (false, true))
    bin2str = Dict((false, false) => 'I', (true, false) => 'X', (true, true) => 'Y', (false, true) => 'Z')

    struct Pauli
    name::String
        bin::Tuple{Vararg{Bool}}

        function Pauli(name::String)
            vec = [str2bin[p] for p in name]
            vec = (Tuple ∘ collect ∘ Iterators.flatten)(vec)
            new(name, vec)
        end

        function Pauli(bin::Tuple{Vararg{Bool}})
            name = join([bin2str[ (bin(ind), bin(ind+1)) ] for ind in 1:2:length(bin) ])
            new(name, bin)
        end

    end

    function Base.:*(a::Pauli, b::Pauli)
        new_bin = a.bin .⊻ b.bin
        Pauli(new_bin)
    end
end

using .Paulis
a = Pauli("X")
b = Pauli("Y")


Pauli("Y", (true, true))

In [9]:
bin = a.bin
bin2str = Dict((false, false) => 'I', (true, false) => 'X', (true, true) => 'Y', (false, true) => 'Z')
[bin2str[ (bin[ind], bin[ind+1]) ] for ind in 1:2:length(bin) ]

1-element Vector{Char}:
 'X': ASCII/Unicode U+0058 (category Lu: Letter, uppercase)

In [4]:
1:2:3

1:2:3

In [5]:
1:2:2

1:2:1

In [8]:
for ind in 1:2:length(bin)
    println(bin[ind])
end

true
